###Unit tests for fastclime.py functions
Author: John Pura                                               
Email: <john.pura@duke.edu>   

In [16]:
%%file tests.py

import fastclime as fc
import numpy as np

############################# 
#Unit tests for is_Hermitian
#############################

#Check that square matrix is successful
def test_square():
    A = np.eye(2)
    assert fc.is_Hermitian(A)

#Check that non-square matrix fails
def test_nonsquare():
    A = np.arange(6).reshape((3,2))
    np.testing.assert_equal(fc.is_Hermitian(A),False)

#Check that square symmetric matrix with missing values is successful
def test_square_miss():
    A = np.eye(3)
    A[1,:] = np.nan
    assert fc.is_Hermitian(A)



Overwriting tests.py


In [17]:
! py.test

============================= test session starts ==============================
platform linux2 -- Python 2.7.9 -- py-1.4.25 -- pytest-2.6.3
collected 0 items 

===============================  in 0.02 seconds ===============================


In [119]:
#Unit tests for fastclime_lambda

import fastclime as fc
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')

L = fastclime.fastclime_generator(n = 10, d = 5)
x = np.array(L.rx2('data'))
Omega = np.array(L.rx2('omega'))

res = fc.fastclime_R(x,0.1,nlambda=50)

Generating data from the multivariate normal distribution with the random graph structure....done.


In [ ]:
############################# 
#Unit tests for is_Hermitian
#############################

#Check that square matrix is successful
def test_square():
    A = np.eye(2)
    assert fc.is_Hermitian(A)

#Check that non-square matrix fails
def test_nonsquare():
    A = np.arange(6).reshape((3,2))
    np.testing.assert_equal(fc.is_Hermitian(A),False)

#Check that square symmetric matrix with missing values is successful
def test_square_miss():
    A = np.eye(3)
    A[1,:] = np.nan
    assert fc.is_Hermitian(A)
    
############################# 
#Unit tests for symmetrize
#############################

#Check that non-square matrix fails
def test_nonsquare():
    A = np.arange(6).reshape((3,2))
    np.testing.assert_raises(TypeError,fc.symmetrize,A)
    
#Check that inputting np.array([1,0],[5,1]) with min rule gives I_2
def test_I2min():
    A = np.array([[1,0],[5,1]],dtype=float)
    np.testing.assert_equal(fc.symmetrize(A,rule="min"),np.eye(2))

#Check that inputting np.array([1,0],[-5,1]) with max rule gives I_2
def test_I2max():
    A = np.array([[1,0],[-5,1]],dtype=float)
    np.testing.assert_equal(fc.symmetrize(A,rule="max"),np.eye(2))

#Check that a positive semi-definite matrix flags user warning
def test_psd():
    Sigma = np.eye(3)
    Omega = np.array([[1,0,0],[0,0,0],[0,0,1]])
    np.testing.assert_warns(UserWarning,fc.loglik,Sigma,Omega)
    
############################# 
#Unit tests for loglik
#############################    
    
#Check that absolute value of log likelihood is <= Inf for at least
#   a positive semidefinite precision matrix
#PosDef Omega
def test_loglikval1():
    np.random.seed(1234)
    A = np.random.normal(0,1,(3,4))
    Sigma = np.corrcoef(A.T)
    Omega = np.linalg.inv(Sigma)
    assert fc.loglik(Sigma,Omega)<=float('Inf')

#PSD Omega
def test_loglikval2():
    Sigma = np.eye(3,dtype=float)
    Omega = np.array([[1,0,0],[0,0,0],[0,0,1]],dtype=float)
    assert fc.loglik(Sigma,Omega)<=float('Inf')
    
#Check that identity matrices (I_3) for both covariance and 
#   precision ma[list()]trices yield correct likelihood value of -3.0
def test_loglikval3():
    Sigma = np.eye(3)
    Omega = np.eye(3)
    np.testing.assert_equal(fc.loglik(Sigma,Omega),-3.0)
    
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')

############################# 
#Unit tests for fastclime_R
#############################

#Generate random data using fastclime.generator in R
L = fastclime.fastclime_generator(n = 200, d = 50)
x = np.array(L.rx2('data'))
Omega = np.array(L.rx2('omega'))

Pyout = fc.fastclime_R(x,0.1,nlambda=50)

Rout = fastclime.fastclime(L.rx2('data'),0.1,50)

#Check that function throws an error when input is not ndarray
def test_input():
    inarray = [[1,2,3],[4,5,6]]
    assert np.testing.assert_raises(TypeError,fc.fastclime_R,inarray)

#For a generated problem, check that the Python output matches that in R

#Check that x is the same
def test_x():
    np.testing.assert_equal(Pyout.x,np.array(Rout.rx2('x')))
    
#Check that cov_input is the same
def test_cov_input():
    np.testing.assert_equal(Pyout.cov_input,np.array(Rout.rx2('cov.input')))

#Check that Sigmahat is the same
def test_Sigmahat():
    np.testing.assert_equal(Pyout.Sigmahat,np.array(Rout.rx2('Sigmahat')))

#Check that maxnlambda is the same
def test_maxnlambda():
    np.testing.assert_equal(Pyout.maxnlambda,np.array(Rout.rx2('maxnlambda')))

#Check that lambdamtx is the same
def test_lambdamtx():
    np.testing.assert_equal(Pyout.lambdamtx,np.array(Rout.rx2('lambdamtx')))

#Check that each icov in icovlist is the same
def test_Sigmahat():
    for i in range(Pyout.maxnlambda):
        np.testing.assert_equal(Pyout.icovlist[:,:,i],np.array(Rout.rx2('icovlist')[i]))
